In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.metrics import mean_squared_error, r2_score

files = [
    "pbmc_cell_frequency_wide.tsv",
    "pbmc_gene_expression_wide_tpm.tsv",
    "plasma_antibody_levels_wide.tsv", 
    "plasma_cytokine_concentrations_by_legendplex_wide.tsv",
    "plasma_cytokine_concentrations_by_olink_wide.tsv",
    "t_cell_activation_wide.tsv",
    "t_cell_polarization_wide.tsv",
]

base_cols = ["age", "infancy_vac", "biological_sex", "Monocytes", "IgG_PT", "ENSG00000277632.1"]

# Load data
subjects = pd.read_csv("training_subject_specimen.tsv", sep="\t")
subjects = subjects[subjects.timepoint >= 0]
subjects = subjects.set_index("specimen_id")
#print(subjects.shape)

for file in files:
    df = pd.read_csv(f"training_{file}", sep='\t')
    df = df.set_index("specimen_id")
    subjects = subjects.merge(df, left_index=True, right_index=True, how="left")
    # print(file, df.shape, subjects.shape)
    
# Clean data
subjects['biological_sex'] = subjects['biological_sex'].map({'Male': 0, 'Female': 1})
subjects['infancy_vac'] = subjects['infancy_vac'].map({'wP': 0, 'aP': 1})
subjects['race'] = subjects['race'].astype('category').cat.codes
subjects['ethnicity'] = subjects['ethnicity'].astype('category').cat.codes

subjects['date_of_boost'] = pd.to_datetime(subjects['date_of_boost'])
subjects['year_of_birth'] = pd.to_datetime(subjects['year_of_birth'])
subjects['age'] = (subjects['date_of_boost'] - subjects['year_of_birth']).dt.days / 365.25

# Split datasets
train_dataset = subjects[subjects.dataset != "2022_dataset"]
test_dataset = subjects[subjects.dataset == "2022_dataset"]

train_dataset = train_dataset.drop(['dataset', 'date_of_boost', 'year_of_birth', 'specimen_type', 'planned_day_relative_to_boost', 'visit', 'actual_day_relative_to_boost'], axis=1)
test_dataset = test_dataset.drop(['dataset', 'date_of_boost', 'year_of_birth', 'specimen_type', 'planned_day_relative_to_boost', 'visit', 'actual_day_relative_to_boost'], axis=1)

print(train_dataset.shape, test_dataset.shape)

# Challenge dataset
csubjects = pd.read_csv("challenge_subject_specimen.tsv", sep="\t")
csubjects = csubjects[csubjects.timepoint >= 0]
csubjects = csubjects.set_index("specimen_id")
#print(subjects.shape)
for file in files:
    df = pd.read_csv(f"challenge_{file}", sep='\t')
    df = df.set_index("specimen_id")
    csubjects = csubjects.merge(df, left_index=True, right_index=True, how="left")
    # print(file, df.shape, subjects.shape)
csubjects['biological_sex'] = csubjects['biological_sex'].map({'Male': 0, 'Female': 1})
csubjects['infancy_vac'] = csubjects['infancy_vac'].map({'wP': 0, 'aP': 1})
csubjects['race'] = csubjects['race'].astype('category').cat.codes
csubjects['ethnicity'] = csubjects['ethnicity'].astype('category').cat.codes

csubjects['date_of_boost'] = pd.to_datetime(csubjects['date_of_boost'])
csubjects['year_of_birth'] = pd.to_datetime(csubjects['year_of_birth'])
csubjects['age'] = (csubjects['date_of_boost'] - csubjects['year_of_birth']).dt.days / 365.25

c_dataset = csubjects.drop(['dataset', 'date_of_boost', 'year_of_birth', 'specimen_type', 'planned_day_relative_to_boost', 'visit', 'actual_day_relative_to_boost'], axis=1)
print(c_dataset.shape)

def generate_dataset(feature, timepoint):
    y_train = train_dataset[train_dataset.timepoint==timepoint][["subject_id", feature]].dropna(subset=feature)
    y_train = y_train.set_index("subject_id")
    y_test = test_dataset[test_dataset.timepoint==timepoint][["subject_id", feature]].dropna(subset=feature)
    y_test = y_test.set_index("subject_id")

    X_train = train_dataset[train_dataset.timepoint==0].drop(['timepoint'], axis=1)
    X_train = X_train[X_train.subject_id.isin(y_train.index)]
    X_train = X_train.set_index("subject_id")

    X_test = test_dataset[test_dataset.timepoint==0].drop(['timepoint'], axis=1)
    X_test = X_test[X_test.subject_id.isin(y_test.index)]
    X_test = X_test.set_index("subject_id")

    nan_count_per_column = X_train.isna().sum()
    cols = nan_count_per_column[nan_count_per_column<50]
    X_train = X_train[cols.index]
    X_test = X_test[cols.index]

    X_test = X_test.fillna(X_train.median())
    X_train = X_train.fillna(X_train.median())

    y_train2 = y_train[feature].values / X_train[feature].values
    y_test2 = y_test[feature].values / X_test[feature].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)
    X_train = pd.DataFrame(X_scaled, columns=X_train.columns)
    X_scaled = scaler.transform(X_test)
    X_test = pd.DataFrame(X_scaled, columns=X_test.columns)

    y_scaler = StandardScaler()
    y_train = y_scaler.fit_transform(y_train).reshape(-1)
    y_test = y_scaler.transform(y_test).reshape(-1)
    y_scaler2 = StandardScaler()
    y_train2 = y_scaler2.fit_transform(y_train2.reshape(-1, 1)).reshape(-1)
    y_test2 = y_scaler2.transform(y_test2.reshape(-1, 1)).reshape(-1)
    # y_train = y_train.values.reshape(-1)
    # y_test = y_test.values.reshape(-1)
    # y_train2 = y_train2.values.reshape(-1)
    # y_test2 = y_test2.values.reshape(-1)
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    
    X_c = csubjects.drop(['timepoint'], axis=1)
    X_c = X_c.set_index("subject_id")
    X_c = X_c[cols.index]

    X_c = X_c.fillna(X_train.median())

    X_scaled = scaler.transform(X_c)
    X_c = pd.DataFrame(X_scaled, columns=X_c.columns)

    print(X_c.shape)
    
    return X_train, y_train, y_train2, X_test, y_test, y_test2, X_c

### Task 1 Models

In [ ]:
X_train, y_train, y_train2, X_test, y_test, y_test2, X_c = generate_dataset(feature = "IgG_PT", timepoint=14)

In [ ]:
ti = np.where(y_train<0)[0]
for a in [0.029]: #t1
    selected_cols = list(X_train.columns)
    last_selected_cols = []
    i = 0
    while sorted(selected_cols) != sorted(last_selected_cols):
        last_selected_cols = selected_cols
        model = LassoLars(alpha=a)  
        model.fit(X_train[selected_cols].iloc[ti], y_train[ti])
        coefficients = model.coef_
        selected_cols = X_train[selected_cols].iloc[ti].columns[coefficients != 0]
        i += 1
    selected_cols = np.unique(list(selected_cols) + base_cols)
    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = LassoLars(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test, y_pred)
    corr, p_value = spearmanr(y_test, y_pred)
    # if p_value < 0.05:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test, y_pred, c=X_test["age"])
    labels = model.predict(X_c_subset)

In [ ]:
ti = np.where(y_train<0)[0]
ti = np.where(y_train2<0)[0]
for a in [0.006]:
    selected_cols = list(X_train.columns)
    last_selected_cols = []
    i = 0
    while sorted(selected_cols) != sorted(last_selected_cols):
        last_selected_cols = selected_cols
        model = LassoLars(alpha=a)  
        model.fit(X_train[selected_cols].iloc[ti], y_train2[ti])
        coefficients = model.coef_
        selected_cols = X_train[selected_cols].iloc[ti].columns[coefficients != 0]
        i += 1
    selected_cols = np.unique(list(selected_cols) + base_cols)

    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train2[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = LassoLars(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test2, y_pred)
    corr, p_value = spearmanr(y_test2, y_pred)
    # if p_value < .1:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test2, y_pred, c=X_test["age"])
    labels2 = model.predict(X_c_subset)

In [ ]:
results =  pd.DataFrame({"subject_id": csubjects.subject_id.values, "1.1) IgG-PT-D14-titer-Rank": labels.argsort()[::-1].argsort() + 1, "1.2) IgG-PT-D14-FC-Rank": labels2.argsort()[::-1].argsort() + 1})
results.sort_values(by="subject_id").to_csv("task1.csv", index=False)

### Task 2 Models

In [ ]:
X_train, y_train, y_train2, X_test, y_test, y_test2, X_c = generate_dataset(feature = "Monocytes", timepoint=1)
correlations = [pearsonr(np.array(X_train.iloc[:,i]), y_train)[0] for i in range(X_train.shape[1])]
correlations = np.nan_to_num(correlations, nan=0)

In [ ]:
ti = np.where(y_train<20)[0]
for a in [0.029]: #t2
# for a in [0.079]: #t2
    N = 5
    selected_features = np.argsort(np.abs(correlations))[-N:]
    selected_cols = np.unique(list(X_train.columns[selected_features]) + base_cols)
    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = LassoLars(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test, y_pred)
    corr, p_value = spearmanr(y_test, y_pred)
    # if p_value < 0.05:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test, y_pred, c=X_test["age"])
    labels = model.predict(X_c_subset)

In [ ]:
ti = np.where(y_train<0.5)[0]
for a in [0.079]:
# for N in range(1, 21):
    N = 5
    selected_features = np.argsort(np.abs(correlations))[-N:]
    selected_cols = np.unique(list(X_train.columns[selected_features]) + base_cols)

    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train2[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = LassoLars(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test2, y_pred)
    corr, p_value = spearmanr(y_test2, y_pred)
    # if p_value < .05:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test2, y_pred, c=X_test["age"])
    labels2 = model.predict(X_c_subset)

In [ ]:
results =  pd.DataFrame({"subject_id": csubjects.subject_id.values, "2.1) Monocytes-D1-Rank": labels.argsort()[::-1].argsort() + 1, "2.2) Monocytes-D1-FC-Rank": labels2.argsort()[::-1].argsort() + 1})
results.sort_values(by="subject_id").to_csv("task2.csv", index=False)

### Task 3 Models

In [ ]:
X_train, y_train, y_train2, X_test, y_test, y_test2, X_c = generate_dataset(feature = "ENSG00000277632.1", timepoint=3)
correlations = [pearsonr(np.array(X_train.iloc[:,i]), y_train)[0] for i in range(X_train.shape[1])]
correlations = np.nan_to_num(correlations, nan=0)

In [ ]:
ti = np.where(y_train<1.)[0]
for a in [0.001]: #t2
    # for N in [5]:
    N = 5
    selected_features = np.argsort(np.abs(correlations))[-N:]
    selected_cols = np.unique(list(X_train.columns[selected_features]) + base_cols)

    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=200)   # LinearRegression()#(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test, y_pred)
    corr, p_value = spearmanr(y_test, y_pred)
    # if p_value < 0.05:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test, y_pred, c=X_test["age"])
    labels = model.predict(X_c_subset)

In [ ]:
ti = np.where(y_train2<10)[0]
for a in [0.006]:
    N=5
    selected_features = np.argsort(np.abs(correlations))[-N:]
    selected_cols = np.unique(list(X_train.columns[selected_features]) + base_cols)

    X_train_subset = X_train[selected_cols].iloc[ti]
    y_train_subset = y_train2[ti]
    X_test_subset = X_test[selected_cols]
    X_c_subset = X_c[selected_cols]
    model = LassoLars(alpha=a)
    model = XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=200)   # LinearRegression()#(alpha=a)
    model.fit(X_train_subset, y_train_subset)
    y_pred = model.predict(X_test_subset)

    mse = mean_squared_error(y_test2, y_pred)
    corr, p_value = spearmanr(y_test2, y_pred)
    # if p_value < .1:
    print(a)
    print(f"\tSpearman: {corr}, {p_value}")
    print(f"\tMean Squared Error (MSE): {mse:.4f}")
    plt.figure()
    plt.scatter(y_test2, y_pred, c=X_test["age"])
    labels2 = model.predict(X_c_subset)

In [ ]:
results =  pd.DataFrame({"subject_id": csubjects.subject_id.values, "3.1) CCL3-D3-Rank": labels.argsort()[::-1].argsort() + 1, "3.2) CCL3-D3-FC-Rank": labels2.argsort()[::-1].argsort() + 1})
results.sort_values(by="subject_id").to_csv("task3.csv", index=False)